In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
asins = pd.read_csv('ASIN.csv',header=None,index_col=False, usecols=[1])
asin = pd.Series(asins[1]).tolist()
tests = []
for i in asin:
    tests.append(r"https://www.amazon.com/exec/obidos/ASIN/" + i)
test = tests

In [3]:
# Function to open web driver
def get_driver():
    options = Options()
    driver = webdriver.Chrome(r"C:\\Users\\wiatr\\anaconda3\\chromedriver.exe", options = options)
    return driver

In [4]:
def parse_url(url):
    driver = get_driver()

   # Open web page of interest & scrape
    driver.get(url)
#     banner = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'productTitle')))
    time.sleep(10)
    htmltext = driver.page_source
    htmltext2 = htmltext[:100]
    driver.close()
    return [url, htmltext]

In [5]:
def main():
    urls = test

    pool = ThreadPool(2)
    records = pool.map(parse_url, urls)
    pool.close()
    pool.join()

    return records

In [6]:
def price_parse(html):
    soup = BeautifulSoup(html, 'html.parser')
    try:
        price = [elem.text for elem in soup.find_all('span', {'class': 'a-offscreen'}) if "$" in elem.text][0]
        return price
    except:
        price = 0
        return price

In [7]:
def product_title(html):
    soup = BeautifulSoup(html, 'html.parser')
    try:
        title = [elem.text for elem in soup.find_all('span', {'id': 'productTitle'})][0].strip()
        return title
    except:
        title = 0
        return title

In [8]:
def product_details(html):
    soup = BeautifulSoup(html, 'html.parser')
    table_names = ['productDetails_detailBullets_sections1','productDetails_detailBullets_sections2','productDetails_techSpec_section_1','productDetails_techSpec_section_2']
    product_details = {}
    try:
        for x in table_names:
            try:
                table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']==x)
                rows=list()
                rows = table.findAll(lambda tag: tag.name=='tr')
                for row in rows:
                    cols=row.find_all('th')
                    frows = row.find_all('td')
                    cols=[x.text.strip() for x in cols]
                    frows = [x.text.strip() for x in frows]
                    product_details[cols[0].encode('ascii', 'ignore').decode("utf-8")] = frows[0].encode('ascii', 'ignore').decode("utf-8")
            except:
                continue
        return product_details
    except:
        return product_details

In [9]:
%%time
if __name__ == '__main__':
    global result
    result = pd.DataFrame(columns = ["url", "html_text"], data = main())

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.647
  (Session info: chrome=96.0.4664.45)


In [10]:
result['price'] = result['html_text'].apply(price_parse)
result['name'] = result['html_text'].apply(product_title)
test_dic = result['html_text'].apply(product_details)
z = list(test_dic)
dfg = pd.DataFrame.from_dict(z)
result2 = pd.concat([result, dfg], axis=1)

NameError: name 'result' is not defined

In [ ]:
result2.head()

In [ ]:
result2.shape

In [ ]:
result2.to_csv('output.csv')

In [ ]:
result2.info()